In [19]:
from pathlib import Path

import vmecpp

Let's load the VMEC++ JSON indata file. We could have just as well used a classic Fortran INDATA file (`input.*`).

In [20]:
vmec_input_filename = Path.cwd() / "data" / "cth_like_fixed_bdy.json"
vmec_input = vmecpp.VmecInput.from_file(vmec_input_filename)

Let's run VMEC++.

In case of errors or non-convergence, a `RuntimeError` is raised.

The output object returned has attributes corresponding
to the usual VMEC outputs: wout, jxbout, mercier, etc.

Hover over `vmecpp.run` to inspect the full signature.

In [26]:
vmec_output = vmecpp.run(vmec_input)


 NS = 15   NO. FOURIER MODES = 41   FTOLV = 1.000e-20   NITER = 25000

 ITER |    FSQR     FSQZ     FSQL    |    fsqr     fsqz      fsql   |   DELT   |  RAX(v=0) |    W_MHD   |   <BETA>   |  <M>  
------+------------------------------+------------------------------+----------+-----------+------------+------------+-------
    1 | 2.33e-01  1.72e-02  5.22e-02 | 2.45e-03  1.97e-04  1.30e-02 | 7.00e-01 | 7.566e-01 | 5.3447e-02 | 1.8384e-03 | 1.027
  200 | 2.16e-08  4.06e-09  3.60e-09 | 1.46e-11  5.70e-12  2.65e-11 | 7.00e-01 | 7.564e-01 | 5.0572e-02 | 1.9280e-03 | 1.035
  400 | 1.62e-10  2.97e-11  6.47e-11 | 4.69e-14  3.93e-14  1.45e-13 | 7.00e-01 | 7.566e-01 | 5.0572e-02 | 1.9285e-03 | 1.040
  600 | 9.54e-13  2.04e-13  1.25e-13 | 5.86e-17  4.51e-17  3.34e-16 | 7.00e-01 | 7.566e-01 | 5.0572e-02 | 1.9285e-03 | 1.040
  800 | 7.01e-15  1.29e-15  1.88e-15 | 1.57e-18  1.05e-18  4.61e-18 | 7.00e-01 | 7.566e-01 | 5.0572e-02 | 1.9285e-03 | 1.040
 1000 | 2.46e-16  5.82e-17  1.27e-16 | 1.22e-19  8.

Now let's perturb the plasma boundary a little bit...

In [22]:
# `rbc` contains Fourier coefficients that parameterize the plasma boundary
vmec_input.rbc[0, 0] *= 0.9
vmec_input.rbc[1, 0] *= 1.1

...and run VMEC++ again, but using its "hot restart" feature:
passing the previously obtained output_quantities ensures that
the run starts already close to the equilibrium, so it will take
fewer iterations to converge this time.

In [23]:
perturbed_output = vmecpp.run(vmec_input, restart_from=vmec_output, verbose=True)


 NS = 15   NO. FOURIER MODES = 41   FTOLV = 1.000e-20   NITER = 25000

 ITER |    FSQR     FSQZ     FSQL    |    fsqr     fsqz      fsql   |   DELT   |  RAX(v=0) |    W_MHD   |   <BETA>   |  <M>  
------+------------------------------+------------------------------+----------+-----------+------------+------------+-------
    1 | 8.45e-07  8.80e-08  2.01e-09 | 4.23e-12  1.28e-12  4.01e-12 | 7.00e-01 | 7.566e-01 | 5.0572e-02 | 1.9285e-03 | 1.040
  200 | 1.04e-16  2.37e-17  3.68e-18 | 8.70e-22  1.51e-21  1.25e-20 | 7.00e-01 | 7.566e-01 | 5.0572e-02 | 1.9285e-03 | 1.040
  400 | 4.56e-19  1.12e-19  7.79e-20 | 3.62e-23  2.46e-23  1.98e-22 | 7.00e-01 | 7.566e-01 | 5.0572e-02 | 1.9285e-03 | 1.040
  539 | 9.81e-21  2.22e-21  2.12e-21 | 9.15e-25  6.70e-25  5.36e-24 | 7.00e-01 | 7.566e-01 | 5.0572e-02 | 1.9285e-03 | 1.040
MHD Energy = 5.057191e-02

NUMBER OF JACOBIAN RESETS = 0
